# Imports

In [1]:
import ydata_profiling
import pandas as pd
import re

# Import of the datasets

In [2]:
nba = pd.read_csv("nba_elo.csv")
pollution = pd.read_csv("pollution_2000_2023.csv", index_col=[0])
us_accidents = pd.read_csv("US_Accidents_March23.csv")
nba_teams = pd.read_csv("team.csv")

In [3]:
# Random Sample because the dataset is to big for profiling
us_accidents_sample = us_accidents.sample(n=10000, random_state=42)

# Profiling of the datasets

## NBA dataset

In [4]:
profile = ydata_profiling.ProfileReport(nba, title="")
profile.to_file("Datasets_profiling/nba_profiling.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Dataset 1: NBA Games (Elo & Carmelo Ratings)

- **Records**: 67,068 | **Variables**: 21  
- **Missing Values**: ~31.4% overall  

### Key Observations:
- The `playoff` column is missing in **93.7%** of rows, significantly limiting any playoff-related comparative analysis.
- `carmelo_*` columns (e.g., `carmelo1_pre`) exhibit over **94% missing data** and may not be reliable for modeling unless imputed.
- High correlation exists among Elo and Carmelo rating variables, suggesting redundancy but offering potential for feature engineering.

### Recommendations:
- Remove or impute `carmelo_*` columns based on Elo scores if needed.
- Consider integrating an external source to tag playoff games accurately.
- Normalize score and rating variables if used in modeling or visual comparison.


## Air Pollution dataset

In [5]:
profile = ydata_profiling.ProfileReport(pollution, title="")
profile.to_file("Datasets_profiling/air_pollution_profiling.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Dataset 2: Pollution Data (2000–2023)

- **Records**: 665,414 | **Variables**: 21  
- **Missing Values**: 0% (Excellent quality)  

### Key Observations:
- Several pollutants (e.g., `CO Mean`, `SO2 Mean`) include **negative values**, which are physically implausible and indicate data entry or measurement errors.
- A high percentage of **zero values** in some columns (e.g., `CO 1st Max Hour`: 46.6%, `SO2 AQI`: 35.2%) may reflect sensor limits or reporting gaps.
- AQI values generally fall within normal ranges; however, extreme values (e.g., `NO2 AQI` up to 153) are present.

### Recommendations:
- Replace negative pollutant values with `NaN` and impute contextually (e.g., by city and month).
- Introduce categorical pollution severity bands (e.g., “Normal”, “Elevated”) based on EPA AQI thresholds.
- Consider excluding high-zero columns or transforming them using binary flags (e.g., "pollutant detected: yes/no").

## NBA teams dataset

In [6]:
profile = ydata_profiling.ProfileReport(nba_teams, title="")
profile.to_file("Datasets_profiling/nba_teams_profiling.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Dataset 3: NBA Teams Metadata

- **Records**: 30 | **Variables**: 7  
- **Missing Values**: 0%  
- **Key Columns**: `abbreviation`, `full_name`, `city`, `state`, `year_founded`

### Key Observations:
- Each row uniquely identifies an NBA team, including location and founding year.
- The `city` and `state` fields provide crucial mapping keys for linking game data to accident or pollution data by location.
- Minor inconsistencies in city naming may arise when joining with external datasets (e.g., `"LA"` vs `"Los Angeles"`).
- `year_founded` could be used for historical context or long-term trend segmentation.

### Recommendations:
- Use `abbreviation` to join with game datasets (`team1`, `team2`).
- Use `state` to align with accident and pollution datasets at the regional level.
- Standardize `city` names if using city-level joins with other datasets.
- Treat this dataset as a reference dimension (lookup table) in data models and star schemas.

## US Traffic Accidents dataset

In [7]:
profile = ydata_profiling.ProfileReport(us_accidents_sample, title="")
profile.to_file("Datasets_profiling/us_accidents_profiling.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## US Accidents Dataset (Sample of 10,000)

## Overview
- **Records**: 10,000  
- **Variables**: 46  
- **Missing Values**: 16,895 cells (≈3.7%)  
- **Duplicates**: None  
- **Size in memory**: 2.7 MiB

## Key Observations
- Variables include location (`Start_Lat`, `Start_Lng`), weather (`Temperature`, `Wind_Speed`, `Precipitation`), and road conditions (e.g., `Bump`, `Traffic_Signal`, `Crossing`).
- Several features contain **missing data**, especially:
  - `End_Lat`, `End_Lng` → 44% missing
  - `Precipitation(in)` → 29.9% missing
  - `Wind_Chill(F)` → 26.8% missing
- Some features are **highly imbalanced** (e.g., `Roundabout`, `Traffic_Calming`, `Railway`, `Give_Way`).
- Several weather-related features contain **zero or negative values**, which may require filtering or correction.
- The `Severity` variable is skewed toward level 2 (≈79%).

## Recommendations
- Consider dropping `End_Lat` and `End_Lng` due to high missing rates, or use them only when available.
- Impute or exclude weather features with >25% missing values based on analytical needs.
- Engineer new features like:
  - `Is_Night`, `Is_Weekend`, or `Time_of_Day`
  - Aggregated road condition indicators
- Normalize weather variables for analysis across cities and states.
- Use boolean features to build binary flags for environmental/road risks.


## Final Notes

- The datasets are generally high quality and well-suited for integrative analysis.
- The most critical issue is **missing and inconsistent values** in weather and playoff-related fields.
- Future efforts should focus on:
  - Cleaning and harmonizing time and location keys across datasets
  - Reducing noise from outliers and negative values
  - Creating derived features that support hypothesis testing and modeling